# init 
importing libs

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    f1_score,
    precision_score,
    recall_score
)

# Reading_cvs

In [2]:
df = pd.read_csv('../data/athletes_sport_details.csv')
country_df = pd.read_csv('../data/country_ranks.csv')
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Sport_Rank,Has_Previous_Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,4,False
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,18,False
2,3,Gunnar Nielsen Aaby,M,24.0,191.0,85.0,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,12,False
3,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,20,False
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,"Speed Skating Women's 1,000 metres",NaN,20,False


# data selection and preprocessing

In [3]:
Gymnastics_df = df[
    (df['Sport'] == 'Gymnastics') &
    (df['Event'].isin([
        "Gymnastics Women's Individual All-Around",
        "Gymnastics Men's Individual All-Around"
    ]))
]

Gymnastics_df.head() , Gymnastics_df.describe()

(      ID                     Name Sex   Age  Height  Weight          Team  \
 32    17  Paavo Johannes Aaltonen   M  28.0   175.0    64.0       Finland   
 40    17  Paavo Johannes Aaltonen   M  32.0   175.0    64.0       Finland   
 107   51        Nstor Abad Sanjun   M  23.0   167.0    64.0         Spain   
 298  244         Raouf Abdelraouf   M  22.0   167.0    63.0         Egypt   
 474  394               Irene Abel   F  19.0   160.0    48.0  East Germany   
 
      NOC        Games  Year  Season            City       Sport  \
 32   FIN  1948 Summer  1948  Summer          London  Gymnastics   
 40   FIN  1952 Summer  1952  Summer        Helsinki  Gymnastics   
 107  ESP  2016 Summer  2016  Summer  Rio de Janeiro  Gymnastics   
 298  EGY  2000 Summer  2000  Summer          Sydney  Gymnastics   
 474  GDR  1972 Summer  1972  Summer          Munich  Gymnastics   
 
                                         Event   Medal  Sport_Rank  \
 32     Gymnastics Men's Individual All-Around  Br

In [4]:
Gymnastics_df_5000_sample = Gymnastics_df[Gymnastics_df["Year"] >= 2000]
Gymnastics_df_5000_sample

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Sport_Rank,Has_Previous_Medal
107,51,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,ESP,2016 Summer,2016,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Individual All-Around,NaN,14,False
298,244,Raouf Abdelraouf,M,22.0,167.0,63.0,Egypt,EGY,2000 Summer,2000,Summer,Sydney,Gymnastics,Gymnastics Men's Individual All-Around,NaN,14,False
485,396,Katja Abel,F,25.0,165.0,55.0,Germany,GER,2008 Summer,2008,Summer,Beijing,Gymnastics,Gymnastics Women's Individual All-Around,NaN,14,False
823,627,Andreea Roxana Acatrinei,F,16.0,150.0,40.0,Romania,ROU,2008 Summer,2008,Summer,Beijing,Gymnastics,Gymnastics Women's Individual All-Around,NaN,14,False
1311,976,Jonna Eva-Maj Adlerteg,F,17.0,153.0,38.0,Sweden,SWE,2012 Summer,2012,Summer,London,Gymnastics,Gymnastics Women's Individual All-Around,NaN,14,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205061,134974,Nataliya Kamilovna Ziganshina,F,18.0,156.0,47.0,Russia,RUS,2004 Summer,2004,Summer,Athina,Gymnastics,Gymnastics Women's Individual All-Around,NaN,14,False
205517,135228,Epke Jan Zonderland,M,22.0,173.0,69.0,Netherlands,NED,2008 Summer,2008,Summer,Beijing,Gymnastics,Gymnastics Men's Individual All-Around,NaN,14,False
205600,135269,Zou Kai,M,20.0,158.0,55.0,China,CHN,2008 Summer,2008,Summer,Beijing,Gymnastics,Gymnastics Men's Individual All-Around,NaN,14,False
205645,135291,Roman Volodymyrovych Zozulia,M,21.0,164.0,59.0,Ukraine,UKR,2000 Summer,2000,Summer,Sydney,Gymnastics,Gymnastics Men's Individual All-Around,NaN,14,False


In [5]:
# Dictionary to store the last cumulative count per name
last_count = {}

# Initialize the column
Gymnastics_df_5000_sample['Count'] = 0

# Sort by Name and Year to ensure proper ordering
Gymnastics_df_5000_sample = Gymnastics_df_5000_sample.sort_values(['Name', 'Year']).reset_index(drop=True)

# Iterate through rows
for name, group in Gymnastics_df_5000_sample.groupby('Name', sort=False):
    # Keep track of cumulative count for this name
    count = 0
    for year, year_group in group.groupby('Year', sort=False):
        # Assign the current count to all rows of this year
        indices = year_group.index
        Gymnastics_df_5000_sample.loc[indices, 'Count'] = count
        # Update cumulative count for next year
        count += len(year_group)

Gymnastics_df_5000_sample = Gymnastics_df_5000_sample.sort_values(by='ID').reset_index(drop=True)
Gymnastics_df_5000_sample

C:\Users\aamyr\AppData\Local\Temp\ipykernel_20160\2131119701.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Gymnastics_df_5000_sample['Count'] = 0


,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Sport_Rank,Has_Previous_Medal,Count
0,51,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,ESP,2016 Summer,2016,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Individual All-Around,NaN,14,False,0
1,244,Raouf Abdelraouf,M,22.0,167.0,63.0,Egypt,EGY,2000 Summer,2000,Summer,Sydney,Gymnastics,Gymnastics Men's Individual All-Around,NaN,14,False,0
2,396,Katja Abel,F,25.0,165.0,55.0,Germany,GER,2008 Summer,2008,Summer,Beijing,Gymnastics,Gymnastics Women's Individual All-Around,NaN,14,False,0
3,627,Andreea Roxana Acatrinei,F,16.0,150.0,40.0,Romania,ROU,2008 Summer,2008,Summer,Beijing,Gymnastics,Gymnastics Women's Individual All-Around,NaN,14,False,0
4,976,Jonna Eva-Maj Adlerteg,F,17.0,153.0,38.0,Sweden,SWE,2012 Summer,2012,Summer,London,Gymnastics,Gymnastics Women's Individual All-Around,NaN,14,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772,134974,Nataliya Kamilovna Ziganshina,F,18.0,156.0,47.0,Russia,RUS,2004 Summer,2004,Summer,Athina,Gymnastics,Gymnastics Women's Individual All-Around,NaN,14,False,0
773,135228,Epke Jan Zonderland,M,22.0,173.0,69.0,Netherlands,NED,2008 Summer,2008,Summer,Beijing,Gymnastics,Gymnastics Men's Individual All-Around,NaN,14,False,0
774,135269,Zou Kai,M,20.0,158.0,55.0,China,CHN,2008 Summer,2008,Summer,Beijing,Gymnastics,Gymnastics Men's Individual All-Around,NaN,14,False,0
775,135291,Roman Volodymyrovych Zozulia,M,21.0,164.0,59.0,Ukraine,UKR,2000 Summer,2000,Summer,Sydney,Gymnastics,Gymnastics Men's Individual All-Around,NaN,14,False,0


making ranking for every country based on economy and technology

In [6]:
x=Gymnastics_df_5000_sample["Team"].unique()
len(x)

73

In [7]:
result = pd.merge(Gymnastics_df_5000_sample, country_df, how="left", on=["Team"])
result = result.drop(columns=["Sport_Rank","Year"])
Gymnastics_df_5000_sample = result[["ID","Name","Sex","Age","Height","Weight","Team","Development_Status","Rank","NOC","Games","Season","City","Sport","Event","Has_Previous_Medal","Count","Medal"]]

In [8]:
Gymnastics_df_5000_sample = Gymnastics_df_5000_sample.dropna(subset=["Development_Status", "Rank"])
Gymnastics_df_5000_sample["ID"] = range(1, len(Gymnastics_df_5000_sample) + 1)
Gymnastics_df_5000_sample.to_csv('../data/Gymnastics_sample.csv')
Gymnastics_df_5000_sample

,ID,Name,Sex,Age,Height,Weight,Team,Development_Status,Rank,NOC,Games,Season,City,Sport,Event,Has_Previous_Medal,Count,Medal
0,1,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,Developed,1,ESP,2016 Summer,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Individual All-Around,False,0,NaN
1,2,Raouf Abdelraouf,M,22.0,167.0,63.0,Egypt,Developing,3,EGY,2000 Summer,Summer,Sydney,Gymnastics,Gymnastics Men's Individual All-Around,False,0,NaN
2,3,Katja Abel,F,25.0,165.0,55.0,Germany,Developed,1,GER,2008 Summer,Summer,Beijing,Gymnastics,Gymnastics Women's Individual All-Around,False,0,NaN
3,4,Andreea Roxana Acatrinei,F,16.0,150.0,40.0,Romania,Developed,1,ROU,2008 Summer,Summer,Beijing,Gymnastics,Gymnastics Women's Individual All-Around,False,0,NaN
4,5,Jonna Eva-Maj Adlerteg,F,17.0,153.0,38.0,Sweden,Developed,1,SWE,2012 Summer,Summer,London,Gymnastics,Gymnastics Women's Individual All-Around,False,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772,773,Nataliya Kamilovna Ziganshina,F,18.0,156.0,47.0,Russia,Upper-Middle,2,RUS,2004 Summer,Summer,Athina,Gymnastics,Gymnastics Women's Individual All-Around,False,0,NaN
773,774,Epke Jan Zonderland,M,22.0,173.0,69.0,Netherlands,Developed,1,NED,2008 Summer,Summer,Beijing,Gymnastics,Gymnastics Men's Individual All-Around,False,0,NaN
774,775,Zou Kai,M,20.0,158.0,55.0,China,Upper-Middle,2,CHN,2008 Summer,Summer,Beijing,Gymnastics,Gymnastics Men's Individual All-Around,False,0,NaN
775,776,Roman Volodymyrovych Zozulia,M,21.0,164.0,59.0,Ukraine,Developing,3,UKR,2000 Summer,Summer,Sydney,Gymnastics,Gymnastics Men's Individual All-Around,False,0,NaN


- we gather some data now lets shift to other processes
- endcoding

In [9]:
Gymnastics_df_5000_sample["Sex"] = Gymnastics_df_5000_sample["Sex"].map({"M": 0, "F": 1})
Gymnastics_df_5000_sample["Has_Previous_Medal"] = Gymnastics_df_5000_sample["Has_Previous_Medal"].map({False: 0, True: 1})
Gymnastics_df_5000_sample["Rank"] = Gymnastics_df_5000_sample["Development_Status"].map({
    "Developed": 3,
    "Upper-Middle": 2,
    "Developing": 1
})

In [10]:
Gymnastics_df_5000_sample.head()

,ID,Name,Sex,Age,Height,Weight,Team,Development_Status,Rank,NOC,Games,Season,City,Sport,Event,Has_Previous_Medal,Count,Medal
0,1,Nstor Abad Sanjun,0,23.0,167.0,64.0,Spain,Developed,3,ESP,2016 Summer,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Individual All-Around,0,0,NaN
1,2,Raouf Abdelraouf,0,22.0,167.0,63.0,Egypt,Developing,1,EGY,2000 Summer,Summer,Sydney,Gymnastics,Gymnastics Men's Individual All-Around,0,0,NaN
2,3,Katja Abel,1,25.0,165.0,55.0,Germany,Developed,3,GER,2008 Summer,Summer,Beijing,Gymnastics,Gymnastics Women's Individual All-Around,0,0,NaN
3,4,Andreea Roxana Acatrinei,1,16.0,150.0,40.0,Romania,Developed,3,ROU,2008 Summer,Summer,Beijing,Gymnastics,Gymnastics Women's Individual All-Around,0,0,NaN
4,5,Jonna Eva-Maj Adlerteg,1,17.0,153.0,38.0,Sweden,Developed,3,SWE,2012 Summer,Summer,London,Gymnastics,Gymnastics Women's Individual All-Around,0,0,NaN


- i use Z-score formula for standardizition (scaling)
- x′=(x−μ)/σ

In [11]:
num_cols = ["Age", "Height", "Weight","Count","Rank","Has_Previous_Medal"]
for col in num_cols:
    mean = Gymnastics_df_5000_sample[col].mean()
    std = Gymnastics_df_5000_sample[col].std()
    Gymnastics_df_5000_sample[col] = (Gymnastics_df_5000_sample[col] - mean) / std
Gymnastics_df_5000_sample = Gymnastics_df_5000_sample.drop(columns=["ID"])
Gymnastics_df_5000_sample.to_csv('../data/Scaled_gymnastics_sample.csv')
Gymnastics_df_5000_sample.describe()

,Sex,Age,Height,Weight,Rank,Has_Previous_Medal,Count
count,777.000000,7.770000e+02,7.770000e+02,7.770000e+02,7.770000e+02,7.770000e+02,777.000000
mean,0.513514,2.926302e-16,1.579746e-15,-1.371704e-16,1.463151e-16,7.315755e-17,0.000000
std,0.500139,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000
min,0.000000,-1.730595e+00,-3.069027e+00,-2.490500e+00,-2.294109e+00,-3.044791e-01,-0.471164
25%,0.000000,-7.590107e-01,-7.039890e-01,-7.832936e-01,-8.245933e-01,-3.044791e-01,-0.471164
50%,1.000000,-3.032292e-02,1.969778e-01,7.030966e-02,6.449227e-01,-3.044791e-01,-0.471164
75%,1.000000,6.983649e-01,7.600820e-01,7.342233e-01,6.449227e-01,-3.044791e-01,-0.471164
max,1.000000,3.370220e+00,2.674637e+00,3.105343e+00,6.449227e-01,3.280071e+00,5.530389


- we check Std to be near 1 

In [12]:
Gymnastics_df_5000_sample

,Name,Sex,Age,Height,Weight,Team,Development_Status,Rank,NOC,Games,Season,City,Sport,Event,Has_Previous_Medal,Count,Medal
0,Nstor Abad Sanjun,0,0.455469,0.760082,0.923913,Spain,Developed,0.644923,ESP,2016 Summer,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Individual All-Around,-0.304479,-0.471164,NaN
1,Raouf Abdelraouf,0,0.212573,0.760082,0.829068,Egypt,Developing,-2.294109,EGY,2000 Summer,Summer,Sydney,Gymnastics,Gymnastics Men's Individual All-Around,-0.304479,-0.471164,NaN
2,Katja Abel,1,0.941261,0.534840,0.070310,Germany,Developed,0.644923,GER,2008 Summer,Summer,Beijing,Gymnastics,Gymnastics Women's Individual All-Around,-0.304479,-0.471164,NaN
3,Andreea Roxana Acatrinei,1,-1.244803,-1.154472,-1.352362,Romania,Developed,0.644923,ROU,2008 Summer,Summer,Beijing,Gymnastics,Gymnastics Women's Individual All-Around,-0.304479,-0.471164,NaN
4,Jonna Eva-Maj Adlerteg,1,-1.001907,-0.816610,-1.542052,Sweden,Developed,0.644923,SWE,2012 Summer,Summer,London,Gymnastics,Gymnastics Women's Individual All-Around,-0.304479,-0.471164,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772,Nataliya Kamilovna Ziganshina,1,-0.759011,-0.478747,-0.688449,Russia,Upper-Middle,-0.824593,RUS,2004 Summer,Summer,Athina,Gymnastics,Gymnastics Women's Individual All-Around,-0.304479,-0.471164,NaN
773,Epke Jan Zonderland,0,0.212573,1.435807,1.398137,Netherlands,Developed,0.644923,NED,2008 Summer,Summer,Beijing,Gymnastics,Gymnastics Men's Individual All-Around,-0.304479,-0.471164,NaN
774,Zou Kai,0,-0.273219,-0.253506,0.070310,China,Upper-Middle,-0.824593,CHN,2008 Summer,Summer,Beijing,Gymnastics,Gymnastics Men's Individual All-Around,-0.304479,-0.471164,NaN
775,Roman Volodymyrovych Zozulia,0,-0.030323,0.422219,0.449689,Ukraine,Developing,-2.294109,UKR,2000 Summer,Summer,Sydney,Gymnastics,Gymnastics Men's Individual All-Around,-0.304479,-0.471164,NaN


In [13]:
print("isnull count:")
print(Gymnastics_df_5000_sample.isna().sum())

isnull count:
Name                    0
Sex                     0
Age                     0
Height                  0
Weight                  0
Team                    0
Development_Status      0
Rank                    0
NOC                     0
Games                   0
Season                  0
City                    0
Sport                   0
Event                   0
Has_Previous_Medal      0
Count                   0
Medal                 747
dtype: int64


# test and train data

- mean and std are near 1 so we can continue
- frist of all we should which data is train and which is test
- next step we use classifaction methods and check accuracy and recall and precision

- i use random.rand that gives me numbers around 0 and 1

In [14]:
msk = np.random.rand(len(Gymnastics_df_5000_sample)) < 0.8
train = Gymnastics_df_5000_sample[msk]
test = Gymnastics_df_5000_sample[~msk]

- data is extreme imbalance due to every year only gets 3 medals and there is only 2 sex (5*3)*2 = 30
- and there is 777 - 747 = 30 so we should oversample or undersample them

- due to this data i prefer to use oversample to make this extreme imbalance faide

In [15]:
medal = train[train["Medal"].notna()]
no_medal = train[train["Medal"].isna()]
medal_oversampled = medal.sample(len(no_medal), replace=True, random_state=42)
df_balanced = pd.concat([medal_oversampled, no_medal])
# for shuffle
df_balanced = (
    pd.concat([medal_oversampled, no_medal])
    .sample(frac=1, random_state=42)
    .reset_index(drop=True)
)
df_balanced.to_csv('../data/Generated_data.csv')
df_balanced

,Name,Sex,Age,Height,Weight,Team,Development_Status,Rank,NOC,Games,Season,City,Sport,Event,Has_Previous_Medal,Count,Medal
0,Yang Wei,0,-0.273219,-0.028264,-0.024535,China,Upper-Middle,-0.824593,CHN,2000 Summer,Summer,Sydney,Gymnastics,Gymnastics Men's Individual All-Around,-0.304479,-0.471164,Silver
1,Shawn Machel Johnson (-East),1,-1.244803,-1.717577,-1.257518,United States,Developed,0.644923,USA,2008 Summer,Summer,Beijing,Gymnastics,Gymnastics Women's Individual All-Around,-0.304479,-0.471164,Silver
2,Dimitri Karbanenko,0,3.370220,1.548428,1.492982,France,Developed,0.644923,FRA,2008 Summer,Summer,Beijing,Gymnastics,Gymnastics Men's Individual All-Around,-0.304479,3.529871,NaN
3,Katheleen Lindor,1,-0.759011,0.760082,0.449689,France,Developed,0.644923,FRA,2008 Summer,Summer,Beijing,Gymnastics,Gymnastics Women's Individual All-Around,-0.304479,-0.471164,NaN
4,Vasiliki Millousi,1,1.669949,-0.591368,-0.878138,Greece,Developed,0.644923,GRE,2012 Summer,Summer,London,Gymnastics,Gymnastics Women's Individual All-Around,-0.304479,1.529353,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1233,Marta Pihan-Kulesza,1,-0.030323,0.196978,-0.119380,Poland,Developed,0.644923,POL,2008 Summer,Summer,Beijing,Gymnastics,Gymnastics Women's Individual All-Around,-0.304479,-0.471164,NaN
1234,Christoph Schrer,0,0.455469,1.548428,1.113602,Switzerland,Developed,0.644923,SUI,2004 Summer,Summer,Athina,Gymnastics,Gymnastics Men's Individual All-Around,-0.304479,-0.471164,NaN
1235,Justin Edward Spring,0,0.698365,1.097945,0.923913,United States,Developed,0.644923,USA,2008 Summer,Summer,Beijing,Gymnastics,Gymnastics Men's Individual All-Around,-0.304479,-0.471164,NaN
1236,Jiang Yuyuan,1,-1.244803,-2.280681,-2.111121,China,Upper-Middle,-0.824593,CHN,2008 Summer,Summer,Beijing,Gymnastics,Gymnastics Women's Individual All-Around,-0.304479,-0.471164,NaN


- to check if there any problem

In [16]:
df_balanced.describe()

,Sex,Age,Height,Weight,Rank,Has_Previous_Medal,Count
count,1238.000000,1238.000000,1238.000000,1238.000000,1238.000000,1238.000000,1238.000000
mean,0.524233,-0.058968,-0.058466,-0.117158,0.005125,0.425171,0.113801
std,0.499614,0.945884,0.950579,0.945385,0.919518,1.443870,1.111920
min,0.000000,-1.730595,-3.069027,-2.490500,-2.294109,-0.304479,-0.471164
25%,0.000000,-0.759011,-0.703989,-0.688449,-0.824593,-0.304479,-0.471164
50%,1.000000,-0.273219,-0.028264,-0.024535,0.644923,-0.304479,-0.471164
75%,1.000000,0.698365,0.534840,0.544534,0.644923,-0.304479,-0.471164
max,1.000000,3.370220,2.674637,3.105343,0.644923,3.280071,5.530389


- Target is a athletic can win or no so we create a column that is our goal
- also we select the features we need for models


In [17]:
df_balanced["Target"] = df_balanced["Medal"].notna().astype(int)
test["Target"] = test["Medal"].notna().astype(int)

# Feature columns
features = [
    "Age",
    "Height",
    "Weight",
    "Sex",
    "Rank",
    "Has_Previous_Medal",
    "Count"
]

X_train = df_balanced[features]
y_train = df_balanced["Target"]

X_test = test[features]
y_test = test["Target"]


C:\Users\aamyr\AppData\Local\Temp\ipykernel_20160\3106028675.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["Target"] = test["Medal"].notna().astype(int)


In [18]:
X_train , y_train , X_test , y_test

(           Age    Height    Weight  Sex      Rank  Has_Previous_Medal  \
 0    -0.273219 -0.028264 -0.024535    0 -0.824593           -0.304479   
 1    -1.244803 -1.717577 -1.257518    1  0.644923           -0.304479   
 2     3.370220  1.548428  1.492982    0  0.644923           -0.304479   
 3    -0.759011  0.760082  0.449689    1  0.644923           -0.304479   
 4     1.669949 -0.591368 -0.878138    1  0.644923           -0.304479   
 ...        ...       ...       ...  ...       ...                 ...   
 1233 -0.030323  0.196978 -0.119380    1  0.644923           -0.304479   
 1234  0.455469  1.548428  1.113602    0  0.644923           -0.304479   
 1235  0.698365  1.097945  0.923913    0  0.644923           -0.304479   
 1236 -1.244803 -2.280681 -2.111121    1 -0.824593           -0.304479   
 1237 -1.244803 -0.478747 -0.498759    1  0.644923           -0.304479   
 
          Count  
 0    -0.471164  
 1    -0.471164  
 2     3.529871  
 3    -0.471164  
 4     1.529353  
 .

- i want to develop types of methods then we'll watch which is better

# KNN

In [19]:
knn = KNeighborsClassifier(
    n_neighbors=5,      
    metric="euclidean", # فاصله
    weights="distance"  # weighted KNN (imbalance)
)

knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

In [20]:

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("F1:", f1_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))

Confusion Matrix:
[[114  14]
 [  1   0]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.89      0.94       128
           1       0.00      0.00      0.00         1

    accuracy                           0.88       129
   macro avg       0.50      0.45      0.47       129
weighted avg       0.98      0.88      0.93       129

F1: 0.0
Precision: 0.0
Recall: 0.0


# ID3

In [23]:
dt = DecisionTreeClassifier(
    criterion='entropy',
    max_depth=5,
    class_weight='balanced',
    random_state=42
)

dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

In [24]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_dt))

print("\nClassification Report:")
print(classification_report(y_test, y_pred_dt))

print("F1:", f1_score(y_test, y_pred_dt))
print("Precision:", precision_score(y_test, y_pred_dt))
print("Recall:", recall_score(y_test, y_pred_dt))

Confusion Matrix:
[[116  12]
 [  1   0]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.91      0.95       128
           1       0.00      0.00      0.00         1

    accuracy                           0.90       129
   macro avg       0.50      0.45      0.47       129
weighted avg       0.98      0.90      0.94       129

F1: 0.0
Precision: 0.0
Recall: 0.0


# GaussianNB

In [25]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred_gnb = gnb.predict(X_test)

In [26]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_gnb))

print("\nClassification Report:")
print(classification_report(y_test, y_pred_gnb))

print("F1:", f1_score(y_test, y_pred_gnb))
print("Precision:", precision_score(y_test, y_pred_gnb))
print("Recall:", recall_score(y_test, y_pred_gnb))

Confusion Matrix:
[[110  18]
 [  1   0]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.86      0.92       128
           1       0.00      0.00      0.00         1

    accuracy                           0.85       129
   macro avg       0.50      0.43      0.46       129
weighted avg       0.98      0.85      0.91       129

F1: 0.0
Precision: 0.0
Recall: 0.0
